## Reading the Grammar file

In [19]:
import re

In [48]:
def read_inst_file(path_file):
  with open(path_file) as file:
    lines = file.readlines()

  variables = None
  initial_var = None
  terminals = None
  production_dictionary = {}
  start_read_productions = False

  for line in lines:
    if not start_read_productions:
      if line.startswith('variaveis'):
        variables = line.strip()
        variables = variables.replace('variaveis:','')
        variables = variables.split(',')

        for i in variables:
          production_dictionary[i] = []
      elif line.startswith('inicial'):
        initial_var = line.strip()
        initial_var = initial_var.replace('inicial:','')
      elif line.startswith('terminais'):
        terminals = line.strip()
        terminals = terminals.replace('terminais:','')
        terminals = terminals.split(',')
      elif line.startswith('producoes'):
        start_read_productions = True
    else:
        line = line.split(': ')
        if line[1][len(line[1]) - 1] == '\n':
          line[1] = line[1][: -1]
        production_dictionary[line[0]].append(line[1])

  return variables, initial_var, terminals, production_dictionary

## Fast Mode

In [54]:
def fast_mode():
  variables, initial_var, terminals, production_dictionary = read_inst_file("test.txt")

  chain = ""
  chain_path = []
  chain_path.append("epsilon")
  epsilon_possibility = None

  for x in production_dictionary:
    for y in production_dictionary[x]:
      if y == "epsilon":
        chain_path.append(x)
        break
    if len(chain_path) != 0:
      break

  i = 1
  while chain_path[i - 1] != initial_var:
    for x in production_dictionary:
      for y in production_dictionary[x]:
        if chain_path[i - 1] in y:
          chain_path.append(x)
          i += 1
          break
      if chain_path[i - 1] == initial_var:
        break

  print("Derivacao:")
  chain_sub_str = []
  chain_sub_str.append(str(chain_path[len(chain_path) - 1]) + " -> ")
  chain += chain_sub_str[0]
  j = 0
  for i in range(len(chain_path)-1, 0, -1):
    for x in production_dictionary[chain_path[i]]:
      if chain_path[i - 1] in x:
        match = re.search(r'(.*?)(?= ->)', chain_sub_str[j])
        if match:
          if "epsilon" in x:
            x = ""
            aux = match.group(0).replace(chain_path[i], x, 1)
            chain_sub_str.append(aux)
          else:
            aux = match.group(0).replace(chain_path[i], x, 1)
            chain_sub_str.append(aux + " -> ")
          j += 1
          chain += chain_sub_str[j]

  print(chain)

  print("Cadeia gerada:")
  print(chain_sub_str[len(chain_sub_str) - 1])

In [61]:
fast_mode()

Derivacao:
S -> bZ -> bdY -> bddX -> bdd
Cadeia gerada:
bdd


## Detailed Mode

In [53]:
def detailed_mode():
  variables, initial_var, terminals, production_dictionary = read_inst_file("test.txt")

  chain_sub_str = initial_var + " -> "
  chain = chain_sub_str
  current_variable = initial_var
  print("Derivacao:")
  print(chain)

  while current_variable != "epsilon":
    print(f"Escolha a operacao de {current_variable}: ")
    print(production_dictionary[current_variable])
    operation = input()
    operacao_valida = False
    for x in production_dictionary[current_variable]:
      if operation in x:
        operacao_valida = True
        break
    if operacao_valida:
      match = re.search(r'(.*?)(?= ->)', chain_sub_str)
      if match:
        if "epsilon" in operation:
          aux = ""
          aux = match.group(0).replace(current_variable, aux, 1)
          chain_sub_str = aux
        else:
          aux = match.group(0).replace(current_variable, operation, 1)
          chain_sub_str = aux + " -> "
        chain += chain_sub_str
        print(chain)
      if operation == "epsilon":
        current_variable = operation
      else:
        for x in variables:
          if x in operation:
            current_variable = x
            break
    else:
      print("Operacao invalida!")

  print("\nCadeia gerada:")
  print(chain_sub_str)

In [56]:
detailed_mode()

Derivacao:
S -> 
Escolha a operacao de S: 
['aX', 'aY', 'bY', 'bZ']
bZ
S -> bZ -> 
Escolha a operacao de Z: 
['S', 'dY', 'epsilon']
epsilon
S -> bZ -> b

Cadeia gerada:
b
